In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from prefect.blocks.system import Secret
from catnip.fla_redshift import FLA_Redshift
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [10]:
df = pd.read_csv("C:\\Users\\riffere\\Desktop\\zane_zips.csv")
df

,zip_code,Owner Postal Code,Owner City,Total Price,Total Sale Quantity
0,00726,00726,Caguas,522.16,2
1,00917,00917,San Juan,246.78,3
2,00926,00926,San Juan,807.64,2
3,01105,01105,Springfield,329.04,4
4,01376,01376,Ciudad de México,261.08,1
...,...,...,...,...,...
545,L7A,L7A3R8,Brampton,383.76,4
546,M3M,M3M2W4,Toronto,246.60,2
547,M9B,M9B5G1,Toronto,653.92,2
548,P1P,P1P1R2,Gravenhurst,164.52,2


In [17]:
q = """
select * from custom.zipcodes_aggregate
"""
df2 = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)
df2

,zip_code,cities,counties,states,country,processed_date
0,A0E,"Baine Harbour, Bay l'Argent, Boat Harbour West...","Burin Peninsula,Clarenville",NL,CA,2023-03-10 12:11:34.209032
1,A0K,"Anchor Point, Baie Verte, Bartletts Harbour, B...","Clarenville,Corner Brook,Labrador City,Wiltond...",NL,CA,2023-03-10 12:11:34.209032
2,A1K,"Bauline, Flatrock, Logy Bay, Middle Cove, Oute...",Conception Bay - St. Johns,NL,CA,2023-03-10 12:11:34.209032
3,A1X,Conception Bay South,Conception Bay - St. Johns,NL,CA,2023-03-10 12:11:34.209032
4,A8A,"Bonne Bay Pond, Cormack, Deer Lake, Pynn's Bro...","Corner Brook,Wiltondale",NL,CA,2023-03-10 12:11:34.209032
...,...,...,...,...,...,...
40651,99811,Sterling,Kenai Peninsula Borough,AK,US,2023-03-10 12:11:34.209032
40652,99827,Elfin cove,Hoonah-Angoon Census Area,AK,US,2023-03-10 12:11:34.209032
40653,99835,Skagway,Skagway Municipality,AK,US,2023-03-10 12:11:34.209032
40654,99923,Coffman cove,Prince of Wales-Hyder Census Area,AK,US,2023-03-10 12:11:34.209032


In [18]:
df = df.merge(df2, on = "zip_code", how = 'left')

In [19]:
df.to_csv('C:\\Users\\riffere\\Desktop\\zipx_final1.csv')

In [3]:
q = """
with a as
    (select event_date, season, marketplace, days_out_bucket, days_out_from_event, 'secondary' as ticket_type, pc_one,
        case
            when pc_one in ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', '1', '2', '3', '4', '5', '6', '7', '8') then 'Lowers'
            when pc_one in ('K', 'L', 'M') then 'Clubs'
            when pc_one in ('N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Y') then 'Uppers'
            when pc_one in ('U', 'V', 'W') then 'Suites'
            when pc_one in ('X','Z') then 'Premuim'
            else 'Null'
        end as location_arena,
        count(*) as paid_seats, sum(gross_revenue) as gross_revenue
    from custom.logitix_v_hockey_sales
    where season in ('2021-22', '2022-23', '2023-24')
    group by event_date, marketplace, season, days_out_bucket, location_arena, days_out_from_event, pc_one),
b as
    (select event_date, season, 'Ticketmaster' as marketplace,
            days_out_bucket, days_out_from_event, 'primary' as ticket_type, pc_one,
        case
            when pc_one in ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', '1', '2', '3', '4', '5', '6', '7', '8') then 'Lowers'
            when pc_one in ('K', 'L', 'M') then 'Clubs'
            when pc_one in ('N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Y') then 'Uppers'
            when pc_one in ('U', 'V', 'W') then 'Suites'
            when pc_one in ('X','Z') then 'Premuim'
            else 'Null'
        end as location_arena,
        sum(paid_seats) as paid_seats, sum(gross_revenue) as gross_revenue
    from custom.cth_historical_all_1718_2223
    where ticket_type = 'Singles'
        and season in ('2021-22', '2022-23')
    group by event_date, season, days_out_bucket, location_arena, days_out_from_event, pc_one),
b2 as
    (select date(event_datetime) as event_date, '2023-24' as season, 'SeatGeek' as marketplace,
            days_out_bucket, days_out_from_event, 'primary' as ticket_type, pc_one,
        case
            when pc_one in ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', '1', '2', '3', '4', '5', '6', '7', '8') then 'Lowers'
            when pc_one in ('K', 'L', 'M') then 'Clubs'
            when pc_one in ('N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Y') then 'Uppers'
            when pc_one in ('U', 'V', 'W') then 'Suites'
            when pc_one in ('X','Z') then 'Premuim'
            else 'Null'
        end as location_arena,
        sum(paid_seats) as paid_seats, sum(gross_revenue) as gross_revenue
    from custom.cth_v_ticket_2324
    where ticket_type = 'Singles'
        and season in ('2023-24')
    group by event_date, season, days_out_bucket, location_arena, days_out_from_event, pc_one),
c as
    (select event_date, abbreviation, tier, start_time_tableau, day_of_week, game_type
    from custom.cth_game_descriptions),
d as
    (select *
     from a
     union
     select *
     from b
     union
     select *
     from b2)
select d.*, abbreviation, tier, start_time_tableau, day_of_week, game_type
from d
left join c on date(d.event_date) = date(c.event_date)
where location_arena is not Null
order by event_date, marketplace, days_out_bucket, location_arena, pc_one
"""
df2 = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)
df2

,event_date,season,marketplace,days_out_bucket,days_out_from_event,ticket_type,pc_one,location_arena,paid_seats,gross_revenue,abbreviation,tier,start_time_tableau,day_of_week,game_type
0,2021-09-26,2021-22,Ticketmaster,A: Day Of,0,primary,7,Lowers,27,270.0000,None,None,None,None,NaN
1,2021-09-26,2021-22,Ticketmaster,A: Day Of,0,primary,8,Lowers,101,1010.0000,None,None,None,None,NaN
2,2021-09-26,2021-22,Ticketmaster,A: Day Of,0,primary,B,Lowers,14,140.0000,None,None,None,None,NaN
3,2021-09-26,2021-22,Ticketmaster,A: Day Of,0,primary,C,Lowers,103,1030.0000,None,None,None,None,NaN
4,2021-09-26,2021-22,Ticketmaster,A: Day Of,0,primary,D,Lowers,66,660.0000,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118511,2024-04-16,2023-24,Vivid Seats,H: 3+ Months Out,100,secondary,J,Lowers,3,344.5650,TOR,B,7:30 PM,Tue,1.0
118512,2024-04-16,2023-24,Vivid Seats,H: 3+ Months Out,241,secondary,J,Lowers,2,202.1448,TOR,B,7:30 PM,Tue,1.0
118513,2024-04-16,2023-24,Vivid Seats,H: 3+ Months Out,259,secondary,O,Uppers,3,231.5421,TOR,B,7:30 PM,Tue,1.0
118514,2024-04-16,2023-24,Vivid Seats,H: 3+ Months Out,127,secondary,P,Uppers,8,742.4376,TOR,B,7:30 PM,Tue,1.0


In [5]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df2, table_name= "emily_test_silver_bullet")